In [ ]:
#@title 1. Installs & Imports
# Install necessary libraries
!pip install diffusers transformers accelerate safetensors invisible_watermark --quiet

import torch
from diffusers import StableDiffusionPipeline, AutoencoderKL
from PIL import Image
from datetime import datetime
import os
import re
from google.colab import drive
import random

# Suppress a common warning from diffusers
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="diffusers.utils.loading_utils")

print("Libraries installed and imported.")

In [ ]:
#@title 2. Configuration

# --- Model & VAE Configuration ---
BASE_MODEL_ID = "stable-diffusion-v1-5/stable-diffusion-v1-5"
# You can use a specific VAE or set to None to use the default from the base model
VAE_MODEL_ID = "stabilityai/sd-vae-ft-mse"

# --- Output Configuration ---
# Path in your Google Drive to save images. Set to None to disable saving to Drive.
# Example: "/content/drive/MyDrive/AI_Images/SD1.5_Generated"
OUTPUT_DIR_GDRIVE = "/content/drive/MyDrive/AI/Images/SD15"

# --- LoRA Configuration ---
# List of LoRAs to load.
# Each LoRA is a dictionary with:
#   "source": "hf" (Hugging Face) or "gdrive" (Google Drive)
#   "id_or_path": Hugging Face model ID or full path in Google Drive
#                 (e.g., "MyLoRAs/your_lora.safetensors" - path relative to "My Drive")
#   "adapter_name": A unique name for this LoRA adapter (used internally by diffusers)
#   "weight": The weight to apply to this LoRA (0.0 to 1.0 typically)
#   "trigger_words": A string of trigger words, comma-separated if multiple.
#                    These will be appended to your prompt.

LORAS_CONFIG = [
    {
        "source": "hf",
        "id_or_path": "OedoSoldier/detail-tweaker-lora", # This is the Detail Tweaker LoRA
        "adapter_name": "add_detail",
        "weight": 0.3, # This LoRA is powerful, start with a lower weight like 0.5
        "trigger_words": "add_detail" # The trigger word for this LoRA
    },
    #{
    #     "source": "gdrive",
    #     "id_or_path": "/content/drive/MyDrive/AI/LoRAs/SD1.5/sample-lora.safetensors",
    #     "adapter_name": "sample_lora",
    #     "weight": 0.6,
    #     "trigger_words": "sample_lora"
    #},
    # Add more LoRAs here if needed
]

# --- Device Configuration ---
if torch.cuda.is_available():
    DEVICE = "cuda"
    torch_dtype = torch.float16 # Use float16 for GPU for faster inference and less memory
else:
    DEVICE = "cpu"
    torch_dtype = torch.float32 # float32 for CPU
print(f"Using device: {DEVICE}")
print(f"Using dtype: {torch_dtype}")
if LORAS_CONFIG:
    print(f"Configured {len(LORAS_CONFIG)} LoRA(s).")
else:
    print("No LoRAs configured.")
if OUTPUT_DIR_GDRIVE:
    print(f"Images will be saved to: {OUTPUT_DIR_GDRIVE}")
else:
    print("Image saving to Google Drive is disabled.")

In [ ]:
#@title 3. Helper Functions

def mount_google_drive():
    """Mounts Google Drive to Colab."""
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
        return True
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return False

def sanitize_filename(text, max_length=60):
    """Sanitizes text for use in filenames."""
    if not text:
        text = "untitled"
    # Remove invalid characters
    text = re.sub(r'[\\/*?:"<>|]', "", text)
    # Replace spaces with underscores
    text = text.replace(" ", "_")
    # Truncate to max_length
    return text[:max_length]

def load_base_pipeline(base_model_id, vae_model_id, device, dtype):
    """Loads the base Stable Diffusion pipeline, optionally with a custom VAE."""
    print(f"Loading base model: {base_model_id}")
    components = {}
    if vae_model_id:
        print(f"Loading custom VAE: {vae_model_id}")
        try:
            vae = AutoencoderKL.from_pretrained(vae_model_id, torch_dtype=dtype)
            components["vae"] = vae
            print("Custom VAE loaded.")
        except Exception as e:
            print(f"Warning: Could not load custom VAE {vae_model_id}. Error: {e}. Using default VAE.")

    try:
        pipeline = StableDiffusionPipeline.from_pretrained(
            base_model_id,
            torch_dtype=dtype,
            **components # Unpack VAE if loaded
        )
        pipeline = pipeline.to(device)
        pipeline.enable_attention_slicing() # Recommended for saving memory
        print("Base pipeline loaded and moved to device.")
        return pipeline
    except Exception as e:
        print(f"Fatal Error: Could not load base model {base_model_id}. Error: {e}")
        raise

def load_and_prepare_loras(pipeline, loras_config_list, device):
    """Loads LoRAs from Hugging Face or Google Drive and prepares them for use."""
    if not loras_config_list:
        print("No LoRAs to load.")
        return []

    loaded_lora_details = []
    gdrive_lora_paths_to_check = []

    for lora_config in loras_config_list:
        source = lora_config.get("source")
        id_or_path = lora_config.get("id_or_path")
        adapter_name = lora_config.get("adapter_name")
        weight = lora_config.get("weight", 0.7) # Default weight if not specified
        trigger_words = lora_config.get("trigger_words", "")

        if not id_or_path or not adapter_name:
            print(f"Skipping LoRA due to missing id_or_path or adapter_name: {lora_config}")
            continue

        print(f"Preparing to load LoRA '{adapter_name}' from {source}: {id_or_path}")

        lora_filename_or_id = id_or_path
        if source == "gdrive":
            # Construct full path for GDrive LoRAs
            full_gdrive_path = os.path.join("/content/drive/MyDrive", id_or_path)
            if not os.path.exists(full_gdrive_path):
                print(f"Warning: LoRA file not found at GDrive path: {full_gdrive_path}. Skipping this LoRA.")
                gdrive_lora_paths_to_check.append(full_gdrive_path) # For a summary warning
                continue
            lora_filename_or_id = full_gdrive_path
            print(f"  Loading LoRA from GDrive path: {lora_filename_or_id}")
        elif source == "hf":
            print(f"  Loading LoRA from Hugging Face ID: {lora_filename_or_id}")
        else:
            print(f"Warning: Unknown LoRA source '{source}' for {adapter_name}. Skipping.")
            continue

        try:
            pipeline.load_lora_weights(
                lora_filename_or_id, # HF model_id or local path
                adapter_name=adapter_name
            )
            loaded_lora_details.append({
                "name": adapter_name,
                "weight": weight,
                "triggers": trigger_words
            })
            print(f"  Successfully loaded LoRA '{adapter_name}'. Weight: {weight}, Triggers: '{trigger_words}'")
        except Exception as e:
            print(f"Error loading LoRA '{adapter_name}' from {lora_filename_or_id}: {e}")

    if gdrive_lora_paths_to_check:
        print("\n--- GDrive LoRA File Check ---")
        print("The following GDrive LoRA paths were configured but the files were not found:")
        for p_check in gdrive_lora_paths_to_check:
            print(f"  - {p_check}")
        print("Please ensure these files exist in your Google Drive at the specified paths.")
        print("--------------------------------\n")


    if loaded_lora_details:
        print(f"\nSuccessfully loaded {len(loaded_lora_details)} LoRA(s).")
    else:
        print("\nNo LoRAs were successfully loaded.")
    return loaded_lora_details


def generate_image(pipeline, prompt, negative_prompt, width, height, num_steps, guidance_scale, seed, active_lora_details_list):
    """Generates an image using the pipeline and applies active LoRAs."""
    print("\n--- Starting Image Generation ---")
    print(f"Prompt: {prompt}")
    if negative_prompt:
        print(f"Negative Prompt: {negative_prompt}")
    print(f"Dimensions: {width}x{height}, Steps: {num_steps}, CFG: {guidance_scale}, Seed: {seed}")

    generator = torch.Generator(device=DEVICE).manual_seed(seed)

    active_adapter_names = [lora['name'] for lora in active_lora_details_list]
    active_adapter_weights = [lora['weight'] for lora in active_lora_details_list]

    if active_adapter_names:
        print(f"Activating LoRAs: {active_adapter_names} with weights: {active_adapter_weights}")
        try:
            # Fusing and unfusing can be more memory efficient for multiple LoRAs
            # but set_adapters is simpler for typical use cases.
            # For many LoRAs, consider fuse/unfuse:
            # pipeline.fuse_lora(adapter_names=active_adapter_names, lora_scale=1.0) # lora_scale here is different from individual weights
            # For simplicity, we'll use set_adapters which respects individual weights directly.
            pipeline.set_adapters(active_adapter_names, adapter_weights=active_adapter_weights)
        except Exception as e:
            print(f"Error setting LoRA adapters: {e}. Proceeding without LoRAs.")
            active_adapter_names = [] # Clear if error
    else:
        print("No LoRAs active for this generation.")


    image = None
    try:
        with torch.autocast(DEVICE if DEVICE == "cuda" else "cpu"): # Autocast for mixed precision on CUDA
            image_result = pipeline(
                prompt=prompt,
                negative_prompt=negative_prompt,
                width=width,
                height=height,
                num_inference_steps=num_steps,
                guidance_scale=guidance_scale,
                generator=generator,
                num_images_per_prompt=1
            )
        image = image_result.images[0]
        print("Image generated successfully.")
    except Exception as e:
        print(f"Error during image generation: {e}")
    finally:
        if active_adapter_names:
            # print("Disabling/resetting LoRAs...")
            # pipeline.unfuse_lora() # if fuse_lora was used
            pipeline.disable_lora() # Simpler, or pipeline.set_adapters([], [])
            # print("LoRAs disabled.")
        pass # No explicit disable needed if set_adapters is used per call and no LoRAs for next

    return image

def save_and_display_image(image, output_dir, base_filename_prompt, generation_params):
    """Saves the image with a timestamped filename and displays it."""
    if not image:
        print("No image to save or display.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    sanitized_prompt = sanitize_filename(base_filename_prompt)
    filename = f"{timestamp}_{sanitized_prompt}.png"

    if output_dir:
        if not os.path.exists(output_dir):
            try:
                os.makedirs(output_dir, exist_ok=True)
                print(f"Created output directory: {output_dir}")
            except Exception as e:
                print(f"Error creating directory {output_dir}: {e}. Image will not be saved to Drive.")
                output_dir = None # Prevent save attempt

        if output_dir: # Check again in case makedirs failed
            save_path = os.path.join(output_dir, filename)
            try:
                image.save(save_path)
                print(f"Image saved to: {save_path}")
            except Exception as e:
                print(f"Error saving image to {save_path}: {e}")

    # Display the image in Colab
    print("\n--- Generated Image ---")
    display(image) # display() is a Colab-specific function for rich output

    print("\n--- Generation Details ---")
    for key, value in generation_params.items():
        print(f"{key.replace('_', ' ').capitalize()}: {value}")
    print("-------------------------")

print("Helper functions defined.")

In [ ]:
#@title 4. Mount Drive (if needed) & Load Models
# Mount Google Drive if an output directory is specified or GDrive LoRAs are configured
needs_drive_mount = bool(OUTPUT_DIR_GDRIVE) or any(lora['source'] == 'gdrive' for lora in LORAS_CONFIG)

if needs_drive_mount:
    print("Google Drive access is required for saving images or loading GDrive LoRAs.")
    if not mount_google_drive():
        print("Warning: Google Drive not mounted. GDrive LoRAs will not load, and images will not be saved to Drive.")
        # If drive mount fails, disable GDrive specific features that depend on it
        if OUTPUT_DIR_GDRIVE:
            print(f"Disabling saving to {OUTPUT_DIR_GDRIVE}.")
            OUTPUT_DIR_GDRIVE = None # Disable saving if mount fails
        # GDrive LoRAs will be skipped by load_and_prepare_loras if path doesn't exist
else:
    print("Google Drive mount not required based on current configuration.")

# --- Load Base Pipeline & VAE ---
try:
    pipeline = load_base_pipeline(BASE_MODEL_ID, VAE_MODEL_ID, DEVICE, torch_dtype)
except Exception as e:
    print(f"Stopping execution due to critical error in loading base pipeline: {e}")
    raise # Or handle more gracefully depending on desired behavior

# --- Load Configured LoRAs ---
if pipeline: # Only proceed if pipeline loaded successfully
    loaded_lora_details = load_and_prepare_loras(pipeline, LORAS_CONFIG, DEVICE)
else:
    loaded_lora_details = []
    print("Skipping LoRA loading as base pipeline failed to load.")

print("\nModel and LoRA loading process complete.")
if not pipeline:
    print("CRITICAL: Pipeline was not loaded. Image generation will not be possible.")
elif not loaded_lora_details and LORAS_CONFIG:
    print("WARNING: Some LoRAs were configured but none were successfully loaded. Check GDrive paths and HF IDs.")

In [ ]:
#@title 5. Image Generation Parameters (Form)
#@markdown ---
#@markdown **Prompting:**
base_prompt_text = "DSLR photography of a black kitten playing with a ball of yarn." #@param {type:"string"}
negative_prompt_text = "bad quality, worse quality, blurry" #@param {type:"string"}
#@markdown ---
#@markdown **Image Dimensions (SD1.5 best works with 512x512 or 512x768, etc.):**
image_width = 512 #@param {type:"slider", min:256, max:1024, step:64}
image_height = 768 #@param {type:"slider", min:256, max:1024, step:64}
#@markdown ---
#@markdown **Generation Settings:**
num_inference_steps = 30 #@param {type:"slider", min:10, max:100, step:1}
guidance_scale = 6 #@param {type:"slider", min:1, max:20, step:0.5}
seed_value = -1 #@param {type:"integer"}
#@markdown *Setting seed to -1 will use a random seed for each generation.*
#@markdown ---
#@markdown **LoRA Usage:**
#@markdown *All successfully loaded LoRAs (from Cell 2 & 4) will be active with their configured weights and trigger words.*
#@markdown *If you want to disable a LoRA, comment it out in the `LORAS_CONFIG` in Cell 2 and re-run Cell 2 and Cell 4.*

print("Parameters form ready. Adjust above and run the next cell to generate.")

In [ ]:
#@title 6. Generate Image!

if 'pipeline' not in globals() or pipeline is None:
    print("Pipeline not loaded. Please run Cell 4 (Load Models) first.")
else:
    # --- Prepare for Generation ---
    # Get parameters from the form (Cell 5)
    current_base_prompt = base_prompt_text
    current_negative_prompt = negative_prompt_text
    current_width = image_width
    current_height = image_height
    current_steps = num_inference_steps
    current_cfg = guidance_scale
    current_seed = seed_value

    if current_seed == -1:
        current_seed = random.randint(0, 2**32 - 1)
        print(f"Using random seed: {current_seed}")

    # Construct the full prompt with LoRA trigger words
    full_prompt_parts = [current_base_prompt]
    if 'loaded_lora_details' in globals() and loaded_lora_details: # Check if var exists and is not empty
        for lora in loaded_lora_details:
            if lora.get("triggers"): # Only add if triggers are defined
                full_prompt_parts.append(lora["triggers"])
    final_prompt = ", ".join(filter(None, full_prompt_parts)) # Join non-empty parts

    # --- Call Generation Function ---
    generated_image = generate_image(
        pipeline,
        prompt=final_prompt,
        negative_prompt=current_negative_prompt,
        width=current_width,
        height=current_height,
        num_steps=current_steps,
        guidance_scale=current_cfg,
        seed=current_seed,
        active_lora_details_list=loaded_lora_details if 'loaded_lora_details' in globals() else []
    )

    # --- Save and Display ---
    if generated_image:
        generation_params_summary = {
            "base_prompt": current_base_prompt,
            "full_prompt_used": final_prompt,
            "negative_prompt": current_negative_prompt,
            "dimensions": f"{current_width}x{current_height}",
            "steps": current_steps,
            "cfg_scale": current_cfg,
            "seed": current_seed,
            "loras_active": [f"{lora['name']} (w:{lora['weight']})" for lora in loaded_lora_details] if 'loaded_lora_details' in globals() and loaded_lora_details else "None"
        }
        save_and_display_image(
            generated_image,
            OUTPUT_DIR_GDRIVE, # Uses the global variable from Cell 2
            current_base_prompt, # For filename
            generation_params_summary
        )
    else:
        print("Image generation failed. No image to display or save.")